## ST2195 Programming for Data Science Coursework Project

Source: 2009 ASA Statistical Computing and Graphics Data Expo

A subset of 2005 and 2006 data would be used to answer the follow questions:

- 1. When is the best time of day, day of the week, and time of year to fly to minimise delays?
- 2. Do older planes suffer more delays?
- 3. How does the number of people flying between different locatiosn change over time?
- 4. Can you detect cascading failures as delays in one airport create delays in another?
- 5. Use the available variables to construct a model that predicts delays.

### Importing libraries

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

### Importing dataset

In [2]:
airport=pd.read_csv('airports.csv')
carrier=pd.read_csv('carriers.csv')
planes_data=pd.read_csv('plane-data.csv')
df_2005=pd.read_csv('2005.csv')
df_2006=pd.read_csv('2006.csv')

# Combining 2005 and 2006 data
df=pd.concat([df_2005,df_2006],ignore_index=True)

### General overview of dataset

In [31]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14282518 entries, 0 to 14282517
Data columns (total 29 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Year               int64  
 1   Month              int64  
 2   DayofMonth         int64  
 3   DayOfWeek          int64  
 4   DepTime            float64
 5   CRSDepTime         int64  
 6   ArrTime            float64
 7   CRSArrTime         int64  
 8   UniqueCarrier      object 
 9   FlightNum          int64  
 10  TailNum            object 
 11  ActualElapsedTime  float64
 12  CRSElapsedTime     float64
 13  AirTime            float64
 14  ArrDelay           float64
 15  DepDelay           float64
 16  Origin             object 
 17  Dest               object 
 18  Distance           int64  
 19  TaxiIn             int64  
 20  TaxiOut            int64  
 21  Cancelled          int64  
 22  CancellationCode   object 
 23  Diverted           int64  
 24  CarrierDelay       int64  
 25  WeatherDelay    

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2005,1,28,5,1603.000,1605,1741.000,1759,UA,541,...,4,23,0,NaN,0,0,0,0,0,0
1,2005,1,29,6,1559.000,1605,1736.000,1759,UA,541,...,6,15,0,NaN,0,0,0,0,0,0
2,2005,1,30,7,1603.000,1610,1741.000,1805,UA,541,...,9,18,0,NaN,0,0,0,0,0,0
3,2005,1,31,1,1556.000,1605,1726.000,1759,UA,541,...,11,10,0,NaN,0,0,0,0,0,0
4,2005,1,2,7,1934.000,1900,2235.000,2232,UA,542,...,5,10,0,NaN,0,0,0,0,0,0


### Analysis & visualization of dataset

In [30]:
pd.set_option("display.max_rows",30)
pd.options.display.float_format='{:.3f}'.format  # Returns float values in 3 decimal places
df.describe().T
df[['NASDelay','SecurityDelay','LateAircraftDelay']].describe()
df[df['AirTime']==0].groupby('UniqueCarrier')['ArrTime'].count()
df[df['Cancelled']==1].groupby('UniqueCarrier')['UniqueCarrier'].count()
df[df['AirTime']==0][['UniqueCarrier','DepTime','CRSDepTime','ArrTime','AirTime']]

,UniqueCarrier,DepTime,CRSDepTime,ArrTime,AirTime
183489,OH,1835.000,1815,2004.000,0.000
785192,OO,645.000,545,701.000,0.000
1351129,OH,2328.000,2150,2335.000,0.000
1359486,OO,2325.000,2110,136.000,0.000
1372300,OO,643.000,630,823.000,0.000
...,...,...,...,...,...
13254398,OO,712.000,708,929.000,0.000
13254673,OO,1814.000,1740,1913.000,0.000
13254712,OO,1557.000,1522,1631.000,0.000
13844227,OO,1401.000,1404,1520.000,0.000


### Q1. When is the best time of day, day of the week, and time of year to fly to minimise delays?